In [1]:
import sqlite3
# on va se connecter à une base de données SQL vide
# SQLite stocke la BDD dans un simple fichier
filepath = "./Universite98459.db"
open(filepath, 'w').close() #crée un fichier vide
CreateDataBase = sqlite3.connect(filepath)

QueryCurs = CreateDataBase.cursor()
QueryCurs.execute("PRAGMA foreign_keys=1")

In [2]:
# On définit une fonction de création de table
def CreateTableTd(nom_bdd):
    QueryCurs.execute('''CREATE TABLE IF NOT EXISTS ''' + nom_bdd + '''
    (Diplome varchar(20) NOT NULL,
       Matiere varchar(20) NOT NULL,
       NoTD int NOT NULL,
       Jour varchar(8) CHECK(jour IN ('Lundi','Mardi','Mercredi','Jeudi','Vendredi','Samedi')),
       Heure varchar(20) NOT NULL,
       Salle int NOT NULL,
       NoEnseignant int NOT NULL,
       constraint PK primary key(NoTD,Matiere,Diplome), /* car les étudiants choisissent un TD et un seul par module */
constraint unicite_TD unique(Heure, Salle,Jour), /* les TD se déroule de façon hebdomadaire */
constraint unicite_TD1 unique (NoEnseignant, Jour, Heure))''')

In [3]:
# On définit une fonction qui permet d'ajouter des observations dans la table    
def AddEntry(nom_bdd, Diplome,Matiere,NoTD,Jour,Heure,Salle,NoEnseignant):
    QueryCurs.execute('''INSERT INTO ''' + nom_bdd + ''' 
    (Diplome,Matiere,NoTD,Jour,Heure,Salle,NoEnseignant) VALUES (?,?,?,?,?,?,?)''',(Diplome,Matiere,NoTD,Jour,Heure,Salle,NoEnseignant))
    
def AddEntries(nom_bdd, data):
    """ data : list with (Diplome,Matiere,NoTD,Jour,Heure,Salle,NoEnseignant) tuples to insert
    """
    QueryCurs.executemany('''INSERT INTO ''' + nom_bdd + ''' 
    (Diplome,Matiere,NoTD,Jour,Heure,Salle,NoEnseignant) VALUES (?,?,?,?,?,?,?)''',data)
    

In [4]:
QueryCurs.execute("DROP TABLE IF EXISTS Inscription")    
QueryCurs.execute("DROP TABLE IF EXISTS Td")    
CreateTableTd('Td')

AddEntry('Td','Licence','BD',1,'Lundi','08H45',232,11111)
AddEntries('Td',
    [('Licence','ALGO',2,'Mercredi','08H35',235,22222),
    ('Licence','ALGO',3,'Lundi','08H45',237,22222),
    ('Master','STAT',5,'Jeudi','08H45',240,44444),
    ('Master','PROBA',4,'Lundi','08H50',240,55555)])

CreateDataBase.commit() # pas de duplication 

In [5]:
QueryCurs.execute('SELECT * FROM Td')  # afficher la table
Values = QueryCurs.fetchall()
print(Values)

[('Licence', 'BD', 1, 'Lundi', '08H45', 232, 11111), ('Licence', 'ALGO', 2, 'Mercredi', '08H35', 235, 22222), ('Licence', 'ALGO', 3, 'Lundi', '08H45', 237, 22222), ('Master', 'STAT', 5, 'Jeudi', '08H45', 240, 44444), ('Master', 'PROBA', 4, 'Lundi', '08H50', 240, 55555)]


In [8]:
import pandas as pd
# méthode SQL Query
df1 = pd.read_sql_query('SELECT * FROM Td', CreateDataBase)
print("En utilisant la méthode read_sql_query \n", df1.head(), "\n")

En utilisant la méthode read_sql_query 
    Diplome Matiere  NoTD      Jour  Heure  Salle  NoEnseignant
0  Licence      BD     1     Lundi  08H45    232         11111
1  Licence    ALGO     2  Mercredi  08H35    235         22222
2  Licence    ALGO     3     Lundi  08H45    237         22222
3   Master    STAT     5     Jeudi  08H45    240         44444
4   Master   PROBA     4     Lundi  08H50    240         55555 



In [9]:
df1.to_csv(r'Td.csv', index = False)   # exporter en csv 

In [460]:
#/*Impossible d'insérer la valeur NULL dans la colonne 'Salle', table 'master.dbo.TD'*/
AddEntry('Td','Licence','STAT',3,'Jeudi','08H45',NULL,44444)

NameError: name 'NULL' is not defined

In [461]:
#/* Valeur de clé primaire dupliquée : ('Master','PROBA','4').*/
AddEntry('Td','Master','PROBA',4,'Jeudi','08H45',240,44444)

IntegrityError: UNIQUE constraint failed: Td.NoEnseignant, Td.Jour, Td.Heure

In [462]:
#/*Valeur de clé unique dupliquée : (11111, Lundi, 08H45). un enseignant ayant 2 cours a donné au meme moment*/
AddEntry('Td','Licence','Stat',1,'Lundi','08H45',237,11111)

IntegrityError: UNIQUE constraint failed: Td.NoEnseignant, Td.Jour, Td.Heure

In [463]:
#/* Valeur de clé dupliquée : (08H50, 240, Lundi).on a deux cours le meme jour a la meme heure et dans la meme salle */
AddEntry('Td','Licence','ALGO',1,'Lundi','08H50',240,33333)

IntegrityError: UNIQUE constraint failed: Td.Heure, Td.Salle, Td.Jour

In [464]:
# On définit une fonction de création de table
def CreateTableInsc(nom_bdd):
    QueryCurs.execute('''CREATE TABLE IF NOT EXISTS ''' + nom_bdd + '''
    (NoEtudiant int  ,
       Diplome varchar(20) NOT NULL,
       Matiere varchar(20) NOT NULL,
       NoTD int NOT NULL,
       Date_inscription date NOT NULL,
       constraint fk_INSCRIPTION Foreign Key (NoTD,Matiere,Diplome) references Td(NoTD,Matiere,Diplome)ON DELETE CASCADE, 

constraint PK2 primary key(NoEtudiant,Diplome,Matiere) )''')

In [465]:
CreateTableInsc('Inscription')

In [466]:
QueryCurs.execute('''INSERT INTO Inscription VALUES (2020825,'Licence', 'BD', 1, '22-10-2020')''')
QueryCurs.execute('''INSERT INTO Inscription VALUES (2020825,'Licence', 'ALGO', 2, '22-10-2020')''')
QueryCurs.execute('''INSERT INTO Inscription VALUES (2020830,'Licence', 'ALGO', 2, '22-11-2020')''')
QueryCurs.execute('''INSERT INTO Inscription VALUES (2020837,'Master', 'STAT', 5, '18-12-2020')''')
QueryCurs.execute('''INSERT INTO Inscription VALUES (2020840,'Master', 'STAT', 5, '18-12-2020')''')


In [467]:
CreateDataBase.commit() # pas de duplication 

In [468]:
QueryCurs.execute('SELECT * FROM Inscription')  # afficher la table
Values = QueryCurs.fetchall()
print(Values)

[(2020825, 'Licence', 'BD', 1, '22-10-2020'), (2020825, 'Licence', 'ALGO', 2, '22-10-2020'), (2020830, 'Licence', 'ALGO', 2, '22-11-2020'), (2020837, 'Master', 'STAT', 5, '18-12-2020'), (2020840, 'Master', 'STAT', 5, '18-12-2020')]


In [469]:

#L'instruction INSERT est en conflit avec la contrainte FOREIGN KEY "fk_INSCRIPTION" de la clé étrangère. 
#Le conflit s'est produit dans la base de données "master", table  CAR il n'existe pas de NoTD=8 dans la table TD*/
QueryCurs.execute('''INSERT INTO Inscription  VALUES (2900830,'Licence', 'PROBA', 8, '22-09-2010')''')




IntegrityError: FOREIGN KEY constraint failed

In [470]:
#/* conflit avec la contrainte FOREIGN KEY "fk_INSCRIPTION car il n'existe pas de matiere VBA avec un numero de TD8 dans la table TD*/
QueryCurs.execute('''INSERT INTO Inscription  VALUES (2900830,'Licence', 'VBA', 8, '22-09-2010')''')

IntegrityError: FOREIGN KEY constraint failed

In [471]:
# conflit avec la contrainte FOREIGN KEY "fk_INSCRIPTION car il n'existe pas de matiere 
# VBA avec un numero de TD8 dans la table TD*/
QueryCurs.execute('''INSERT INTO Inscription  VALUES  (2900830,'Licence', 'VBA', 8, '22-09-2010')''')

IntegrityError: FOREIGN KEY constraint failed

In [472]:
# conflit avec la contrainte FOREIGN KEY "fk_INSCRIPTION car il n'existe pas de diplome Doctorat dans la table Td*/
QueryCurs.execute('''INSERT INTO Inscription  VALUES  (2900830,'Doctorat', 'Stat', 8, '18-09-2010')''')

IntegrityError: FOREIGN KEY constraint failed

In [473]:
# Violation de la contrainte primary key  Valeur de clé primaire dupliquée : (2020825, Licence, ALGO). et aussi (2020825, Master,STAT) */
#/*update INSCRIPTION set NoEtudiant=2020815;*/
QueryCurs.execute('''INSERT INTO Inscription  VALUES  (2020825, 'Licence', 'ALGO', 2, '18-09-2010')''')


IntegrityError: UNIQUE constraint failed: Inscription.NoEtudiant, Inscription.Diplome, Inscription.Matiere

In [474]:
QueryCurs.execute('SELECT * FROM Inscription')  # afficher la table
Values = QueryCurs.fetchall()
print(Values)

[(2020825, 'Licence', 'BD', 1, '22-10-2020'), (2020825, 'Licence', 'ALGO', 2, '22-10-2020'), (2020830, 'Licence', 'ALGO', 2, '22-11-2020'), (2020837, 'Master', 'STAT', 5, '18-12-2020'), (2020840, 'Master', 'STAT', 5, '18-12-2020')]


In [475]:
QueryCurs.execute('SELECT * FROM Td')  # afficher la table
Values = QueryCurs.fetchall()
print(Values)

[('Licence', 'BD', 1, 'Lundi', '08H45', 232, 11111), ('Licence', 'ALGO', 2, 'Mercredi', '08H35', 235, 22222), ('Licence', 'ALGO', 3, 'Lundi', '08H45', 237, 22222), ('Master', 'STAT', 5, 'Jeudi', '08H45', 240, 44444), ('Master', 'PROBA', 4, 'Lundi', '08H50', 240, 55555)]


In [478]:
QueryCurs.execute("""SELECT * FROM Td WHERE  NoTD=1""") .fetchall()

[('Licence', 'BD', 1, 'Lundi', '08H45', 232, 11111)]

In [479]:
QueryCurs.execute("""SELECT * FROM Td WHERE  Diplome='Licence'""") .fetchall()

[('Licence', 'BD', 1, 'Lundi', '08H45', 232, 11111),
 ('Licence', 'ALGO', 2, 'Mercredi', '08H35', 235, 22222),
 ('Licence', 'ALGO', 3, 'Lundi', '08H45', 237, 22222)]